###Задание

1) Запустить ячейку ниже, чтобы получить 2 жанра. Для полученных жанров провести все этапы предварительной обработки текста (как в примере), обучить наивный байесовский классификатор, численно оценить его работу.

In [ ]:
 import random 
 lst = ['Christian', 'Country', 'Pop', 'Rock', 'R&B'] 
 print('ваши жанры', random.choice(lst), 'и', random.choice(lst)) 

ваши жанры Pop и R&B


[Выпали Pop и R&B]

Для улучшения качества обучения модели можно попробовать избавиться от знаков препинания и проследить, чтобы процентное соотношение песен обоих жанров было примерно 50 на 50

2) Найти (нагуглить) по песне каждого из жанров, которые Вам достались, после необходимой обработки их текстов определить жанр обеих песен с помощью обученной в ходе выполнения предыдущего пункта модели.

3) С помощью набора данных по ссылке аналогичным образом научить модель отличать тексты песен Дэвида Боуи от текстов песен Пола МакКартни 
https://www.kaggle.com/italomarcelo/dataset-lyrics-music-mini

<h1> Лабораторная 2 </h1>

# 1 Задание

Импорт модулей и подключение к гугл диску

In [ ]:
import pandas as pd
import numpy as np
import random
import nltk
import sklearn

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

from sklearn.metrics import * 
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB

from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer 

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


Инициализация

In [ ]:
downloader = drive.CreateFile({'id':'1zY_BzCVPLpdrMbuJ5VkFG0nWp4sA_bYb'})
downloader.GetContentFile('dataset.csv')

data = pd.read_csv("dataset.csv")
data.head()

,genre,lyrics,SongInfo
0,Christian,"Who am I, that the Lord of all the earth Woul...",CASTING CROWNS - WHO AM I LYRICS
1,Christian,Glory Revealed By His Wounds He was pierced ...,GLORY REVEALED - BY HIS WOUNDS LYRICS
2,Christian,Lord of heaven and earth Lord of all creation...,CAEDMON'S CALL - GOD OF WONDERS LYRICS
3,Christian,I can only imagine what it will be like When ...,MERCYME - I CAN ONLY IMAGINE LYRICS
4,Christian,I am not skilled to understand What God has w...,AARON SHUST - MY SAVIOR MY GOD LYRICS


Выбираем жанры, приводим к lower-кейсу, убираем стоп-слова, токенизируем


In [ ]:

columns = data[['genre', 'lyrics']]
columns = columns[(columns.genre == 'Pop') | (columns.genre == 'R&B')]

lowered = columns['lyrics'].str.lower()
columns['lowered'] = lowered

tokened = columns.apply(lambda row: nltk.word_tokenize(row['lowered']), axis=1)
columns['tokened'] = tokened

#print(stopwords.words('english'))
noise = stopwords.words('english')
withoutstop = columns['tokened'].apply(lambda x: [item for item in x if item not in noise])
without_stop = []
for a in withoutstop:    
    without_stop.append(", ".join(a))
columns['without_stop'] = without_stop

Лемматизация

In [ ]:
lemmatizer = WordNetLemmatizer()
lemmatized = columns['without_stop'].apply(lambda x: [lemmatizer.lemmatize(x)])
lemma = []
for a in lemmatized:    
    lemma.append(", ".join(a))
columns['lemmatized'] = lemma

#columns

Разделим данные на обучающую и тестовые выборки

x_train - тексты, на которых мы обучаем модель. В данном случае мы используем столбец lemmatized, так как он содержит данные, прошедшие все этапы подготовки

y_train - жанры, соответствующие текстам, на которых модель обучается - столбец genre

x_test - точно такие же тексты из набора данных, на которых мы будем проверять, насколько модель научилась предсказывать жанр

y_test - жанры, соответствующие x_test. т.е. мы смотрим, насколько предсказания соответствуют содержимому этой переменной, чтобы оценить качество обучения

Recall демонстрирует способность алгоритма обнаруживать данный класс вообще, 
precision — способность отличать этот класс от других классов,
 f1-score - среднее гармоническое precision и recall

In [ ]:
# ~ 2/3 для обучения и ~ 1/3 для тестирования
x_train, x_test, y_train, y_test = train_test_split(columns.lemmatized, columns.genre, train_size = 0.7)


vectorizer = CountVectorizer(ngram_range=(1, 3))
vectorized_x_train = vectorizer.fit_transform(x_train)

clf = MultinomialNB()
clf.fit(vectorized_x_train, y_train)
# тестовую выборку просто векторизировали
vectorized_x_test = vectorizer.transform(x_test)

# print(vectorized_x_test)
clf.predict(vectorized_x_test)

pred = clf.predict(vectorized_x_test)
print(classification_report(y_test, pred))

              precision    recall  f1-score   support

         Pop       0.69      0.55      0.61        33
         R&B       0.53      0.68      0.60        25

    accuracy                           0.60        58
   macro avg       0.61      0.61      0.60        58
weighted avg       0.62      0.60      0.60        58



# 2 Задание

Загрузка датасета

In [ ]:
downloader = drive.CreateFile({'id':'17yy2-NZKyqZ57Q9RVCCy0HVCx66CKLuk'})
downloader.GetContentFile('MyDataSet.csv')

data = pd.read_csv("MyDataSet.csv")
data.head()


,genre,lyrics,SongInfo
0,R&B,yeah you know what i like the players no diggi...,Blackstreet – No Diggity
1,Pop,i stay out too late got nothing in my brain th...,Taylor Swift – Shake It Off


Выбираем жанры, приводит к lower-кейсу, убираем стоп-слова, токенизируем




In [ ]:
columns = data[['genre', 'lyrics']]
columns = columns[(columns.genre == 'Pop') | (columns.genre == 'R&B')]

lowered = columns['lyrics'].str.lower()
columns['lowered'] = lowered

tokened = columns.apply(lambda row: nltk.word_tokenize(row['lowered']), axis=1)
columns['tokened'] = tokened

#print(stopwords.words('english'))
noise = stopwords.words('english')
withoutstop = columns['tokened'].apply(lambda x: [item for item in x if item not in noise])
without_stop = []
for a in withoutstop:    
    without_stop.append(", ".join(a))
columns['without_stop'] = without_stop

Лемматизация

In [ ]:
lemmatizer = WordNetLemmatizer()
lemmatized = columns['without_stop'].apply(lambda x: [lemmatizer.lemmatize(x)])
lemma = []
for a in lemmatized:    
    lemma.append(", ".join(a))
columns['lemmatized'] = lemma

columns

,genre,lyrics,lowered,tokened,without_stop,lemmatized
0,R&B,yeah you know what i like the players no diggi...,yeah you know what i like the players no diggi...,"[yeah, you, know, what, i, like, the, players,...","yeah, know, like, players, diggity, ,, doubt, ...","yeah, know, like, players, diggity, ,, doubt, ..."
1,Pop,i stay out too late got nothing in my brain th...,i stay out too late got nothing in my brain th...,"[i, stay, out, too, late, got, nothing, in, my...","stay, late, got, nothing, brain, 's, people, s...","stay, late, got, nothing, brain, 's, people, s..."


Создаем тестовую выборку и векторизиурем ее

In [ ]:
x_test = columns['lemmatized']
vectorized_x_test = vectorizer.transform(x_test)
clf.predict(vectorized_x_test)

array(['Pop', 'R&B'], dtype='<U3')

Песни получились жанра R&B, это говорит о небольшой точности НБК
(После повторого запуска всех ячеек значение уже другое, теперь первая песня - Pop, вторая - R&B)

Посмотрим что в классификации и увидим, что у Pop "recall" (способность находить её) действительно 0

In [ ]:
y_test = columns['genre']
vectorized_y_test = vectorizer.transform(y_test)
print(classification_report(y_test, clf.predict(vectorized_x_test)))

              precision    recall  f1-score   support

         Pop       0.00      0.00      0.00       1.0
         R&B       0.00      0.00      0.00       1.0

    accuracy                           0.00       2.0
   macro avg       0.00      0.00      0.00       2.0
weighted avg       0.00      0.00      0.00       2.0



# Задание 3

Инициализация

In [ ]:
downloader = drive.CreateFile({'id':'1jJ6bH9Xhp4-DmkE9ZihtNqeanuWJNVcW'})
downloader.GetContentFile('dataset-lyrics-musics-mini.csv')

data = pd.read_csv("dataset-lyrics-musics-mini.csv")
data.head()

,cantorId,cantorNome,musicaNome,letra
0,0,david-bowie,Heroes,"I, I will be king. And you, you will be queen...."
1,0,david-bowie,Starman,"Didn't know what time it was,. The lights were..."
2,0,david-bowie,Space Oddity,Ground control to Major Tom. Ground control to...
3,0,david-bowie,Life On Mars?,It's a god-awful small affair. To the girl wit...
4,0,david-bowie,Modern Love,I know when to go out. And when to stay in. Ge...


Выбираем певцов, приводим к lower-кейсу, убираем стоп-слова, токенизируем




In [ ]:
columns = data[['cantorNome', 'letra']]
columns = columns[(columns.cantorNome == 'david-bowie') | (columns.cantorNome == 'paul-mccartney')]

lowered = columns['letra'].str.lower()
columns['lowered'] = lowered

tokened = columns.apply(lambda row: nltk.word_tokenize(row['lowered']), axis=1)
columns['tokened'] = tokened

noise = stopwords.words('english')
withoutstop = columns['tokened'].apply(lambda x: [item for item in x if item not in noise])
without_stop = []
for a in withoutstop:    
    without_stop.append(", ".join(a))
    
columns['without_stop'] = without_stop


Лемматизация

In [ ]:
lemmatized = columns['without_stop'].apply(lambda x: [lemmatizer.lemmatize(x)])
lemma = []
for a in lemmatized:    
    lemma.append(", ".join(a))
    
columns['lemmatized'] = lemma

Разделим данные на обучающую и тестовые выборки

x_train - тексты, на которых мы обучаем модель. В данном случае мы используем столбец lemmatized, так как он содержит данные, прошедшие все этапы подготовки

y_train - жанры, соответствующие текстам, на которых модель обучается - столбец genre

x_test - точно такие же тексты из набора данных, на которых мы будем проверять, насколько модель научилась предсказывать жанр

y_test - жанры, соответствующие x_test. т.е. мы смотрим, насколько предсказания соответствуют содержимому этой переменной, чтобы оценить качество обучения

Recall демонстрирует способность алгоритма обнаруживать данный класс вообще, precision — способность отличать этот класс от других классов, f1-score - среднее гармоническое precision и recall

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(columns.lemmatized, columns.cantorNome, train_size = 0.7)

vectorizer = CountVectorizer(ngram_range=(1, 3))
vectorized_x_train = vectorizer.fit_transform(x_train)


Наиивный байесовский классификатор

In [ ]:
clf = MultinomialNB()
clf.fit(vectorized_x_train, y_train)
vectorized_x_test = vectorizer.transform(x_test)
clf.predict(vectorized_x_test)

array(['david-bowie', 'paul-mccartney', 'paul-mccartney',
       'paul-mccartney', 'paul-mccartney', 'paul-mccartney',
       'paul-mccartney', 'david-bowie', 'paul-mccartney',
       'paul-mccartney', 'david-bowie', 'david-bowie', 'david-bowie',
       'david-bowie', 'david-bowie', 'david-bowie', 'paul-mccartney',
       'paul-mccartney', 'david-bowie', 'david-bowie', 'paul-mccartney',
       'david-bowie', 'david-bowie', 'paul-mccartney', 'paul-mccartney',
       'david-bowie', 'david-bowie', 'david-bowie', 'david-bowie',
       'david-bowie', 'paul-mccartney', 'david-bowie', 'david-bowie',
       'david-bowie', 'paul-mccartney', 'paul-mccartney', 'david-bowie',
       'paul-mccartney', 'david-bowie', 'david-bowie', 'paul-mccartney',
       'david-bowie', 'paul-mccartney', 'paul-mccartney', 'david-bowie',
       'david-bowie', 'david-bowie', 'paul-mccartney', 'david-bowie',
       'david-bowie', 'david-bowie', 'david-bowie', 'paul-mccartney',
       'david-bowie', 'david-bowie', 'pau

И, собственно, оценки классификации:

In [ ]:
print(classification_report(y_test, clf.predict(vectorized_x_test)))

                precision    recall  f1-score   support

   david-bowie       0.77      0.76      0.76       147
paul-mccartney       0.75      0.75      0.75       138

      accuracy                           0.76       285
     macro avg       0.76      0.76      0.76       285
  weighted avg       0.76      0.76      0.76       285

